<a href="https://colab.research.google.com/github/Dzeno0405/AI-ML-testing/blob/main/testingDzenan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("HamidRezaAttar/gpt2-product-description-generator")
model = AutoModelForCausalLM.from_pretrained("HamidRezaAttar/gpt2-product-description-generator")
gen = pipeline("text-generation", model=model, tokenizer=tokenizer, config={"max_length": 100})

print(gen("Introducing our new ergonomic office chair: "))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Introducing our new ergonomic office chair: \xa0a sophisticated contemporary design at an affordable price. \xa0this ergonomic office chair features a strong, hardwood frame with a white foam cushion top and back. \xa0the ergonomic one touch pneumatic seat height adjustment and the adjustable tilt tension mechanism ensure total comfort and flexibility. \xa0the chair is ergonomically designed for a comfortable and secure position. \xa0attractive chrome finish and non-skid foot pads add to the modern look. \xa0The smooth, rounded and rounded armsrestful\xa0are the perfect complement to the ergonomic design and comfort of the office chair. \xa0the ergonomic and easy to use design makes it a great chair for any office. \xa0the large, contoured seat provides maximum comfort and support so you can enjoy total relaxation. \xa0the 360 degree swivel seat and back support makes for a strong, comfortable seat. \xa0the chair is fully assembled and easily folds for convenient s

In [2]:
pip install transformers datasets accelerate


In [3]:
from google.colab import files
uploaded = files.upload()


Saving quantity_offers_dataset.csv to quantity_offers_dataset (1).csv


In [8]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

# STEP 1: Load tokenizer ONCE and add PAD token
tokenizer = AutoTokenizer.from_pretrained("HamidRezaAttar/gpt2-product-description-generator")

# Add PAD token and update tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# STEP 2: Load and format dataset
df = pd.read_csv("quantity_offers_dataset.csv")

# Combine product + offer into one field for language modeling
def format_example(row):
    return {
        "text": f"Product: {row['input_text']}\nOffer: {row['target_text']}"
    }

data = df.apply(format_example, axis=1).tolist()
dataset = Dataset.from_list(data)

# STEP 3: Tokenize dataset
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize)

# STEP 4: Load model and resize embeddings to fit tokenizer
model = AutoModelForCausalLM.from_pretrained("HamidRezaAttar/gpt2-product-description-generator")
model.resize_token_embeddings(len(tokenizer))

print("✅ Dataset tokenized and model updated.")


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


✅ Dataset tokenized and model updated.


In [10]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

# Check for MPS (Apple Silicon) or fallback to CPU
device = "mps" if torch.backends.mps.is_available() else "cpu"
model.to(device)

# Data collator handles padding for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training config
training_args = TrainingArguments(
    output_dir="./gpt2-quantity-offers",         # Where to save model
    overwrite_output_dir=True,
    per_device_train_batch_size=2,               # Feel free to adjust
    num_train_epochs=5,                          # 3–5 for small dataset
    learning_rate=5e-5,
    logging_steps=10,
    save_steps=50,
    report_to="none",                            # Disable wandb
    fp16=False,                                  # No mixed precision for MPS
)

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()


/tmp/ipython-input-10-61786826.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,3.404200
20,2.195000
30,1.649000
40,1.350100
50,1.233700
60,0.839300
70,0.957800
80,0.650600
90,0.578500
100,0.585600


TrainOutput(global_step=125, training_loss=1.173824659347534, metrics={'train_runtime': 779.3079, 'train_samples_per_second': 0.321, 'train_steps_per_second': 0.16, 'total_flos': 16330752000000.0, 'train_loss': 1.173824659347534, 'epoch': 5.0})

In [11]:
model.save_pretrained("./gpt2-quantity-offers")
tokenizer.save_pretrained("./gpt2-quantity-offers")


('./gpt2-quantity-offers/tokenizer_config.json',
 './gpt2-quantity-offers/special_tokens_map.json',
 './gpt2-quantity-offers/vocab.json',
 './gpt2-quantity-offers/merges.txt',
 './gpt2-quantity-offers/added_tokens.json',
 './gpt2-quantity-offers/tokenizer.json')

In [12]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model="./gpt2-quantity-offers",
    tokenizer=tokenizer,
    device=0 if torch.backends.mps.is_available() else -1
)

prompt = "Product: Wireless charging pad with dual device support\nOffer:"
result = generator(prompt, max_new_tokens=40)[0]['generated_text']
print(result)



Device set to use cpu


Product: Wireless charging pad with dual device support
Offer: Buy 1 for $89.99 • 2 for $169.99 • 3 for $219.99 • 4 for $219.99 • 5 for $219.99 • 6 for $219


# Model successfully learned the product-to-offer format, generating realistic multi-tier pricing with proper formatting and believable values. To improve, add more diverse training data with clear stopping tokens and punctuation, as well as examples that help it better capture volume discount patterns and avoid repetitive outputs.


In [13]:
base = 29.99
dataset_entry = {
    "input_text": "Premium USB-C Cable, 2m length, braided",
    "target_text": f"Buy 1 for ${base:.2f} • 2 for ${base*1.8:.2f} • 3 for ${base*2.3:.2f}"
}


In [14]:
result = generator(prompt, max_new_tokens=40, eos_token_id=tokenizer.encode("\n")[0])[0]['generated_text']
